# Загрузка данных анкет и проведение МАИ, анализ полученных результатов
Данные: заполненные анкеты экспертов (таблицы Excel) 

In [18]:
import re
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
from random import randint

In [19]:

df = pd.read_excel(r'C:\Users\user\матрица\Экс1.xlsx')
df.head()

,Вид происшествия,Аварии,Инциденты,Предпосылки
0,Аварии,1.000000,7.000,9
1,Инциденты,0.142857,1.000,8
2,Предпосылки к инцидентам,0.111111,0.125,1


In [20]:
print('Число строк и столбцов в наборе данных:\n', df.shape)

Число строк и столбцов в наборе данных:
 (3, 4)


In [21]:
# типы столбцов 
df.dtypes

Вид происшествия     object
Аварии              float64
Инциденты           float64
Предпосылки           int64
dtype: object

In [22]:
#удаление столбца типа object для дальнейшей работы с фреймом 
df.pop('Вид происшествия') 
print(df)

     Аварии  Инциденты  Предпосылки
0  1.000000      7.000            9
1  0.142857      1.000            8
2  0.111111      0.125            1


In [23]:
# типы столбцов 
df.dtypes

Аварии         float64
Инциденты      float64
Предпосылки      int64
dtype: object

In [24]:
# также меням тип данных Предпосылки
df['Предпосылки'] = df['Предпосылки'].astype(float)

In [25]:
df.loc[:, df.columns[df.dtypes == 'float64']].head()

,Аварии,Инциденты,Предпосылки
0,1.000000,7.000,9.0
1,0.142857,1.000,8.0
2,0.111111,0.125,1.0


In [26]:
DF_1=df[:].copy()

In [27]:
#Найдем сумму всех столбцов, для того чтобы в дальнейшем пересчитать полученные значения
df.sum ()

Аварии          1.253968
Инциденты       8.125000
Предпосылки    18.000000
dtype: float64

In [29]:
# Считаем значения для нашей матрицы, путем деления каждого значения в столбце на сумму столбца
df['new_Аварии'] = df['Аварии'].astype(float).div(df['Аварии'].sum())
df['new_Инциденты'] = df['Инциденты'].astype(float).div(df['Инциденты'].sum())
df['new_Предпосылки'] = df['Предпосылки'].astype(float).div(df['Предпосылки'].sum())

In [30]:
df.head()

,Аварии,Инциденты,Предпосылки,new_Аварии,new_Инциденты,new_Предпосылки
0,1.000000,7.000,9.0,0.797468,0.861538,0.500000
1,0.142857,1.000,8.0,0.113924,0.123077,0.444444
2,0.111111,0.125,1.0,0.088608,0.015385,0.055556


In [33]:
# Получили новую таблицу теперь отделим значения нашей матрицы от основной таблицы
df_new = df[['new_Аварии', 'new_Инциденты', 'new_Предпосылки']]
df_new

,new_Аварии,new_Инциденты,new_Предпосылки
0,0.797468,0.861538,0.500000
1,0.113924,0.123077,0.444444
2,0.088608,0.015385,0.055556


In [34]:
#Вычислим средние значение по строке и запишем полученный результот в mean
df_new['mean'] = df_new.mean(axis=1)
df_new.head()

C:\Users\user\anaconda3\envs\envpy3.5\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,new_Аварии,new_Инциденты,new_Предпосылки,mean
0,0.797468,0.861538,0.500000,0.719669
1,0.113924,0.123077,0.444444,0.227148
2,0.088608,0.015385,0.055556,0.053183


In [35]:
#проверка, среднее значение в сумме должно быть равно 1
df_new['mean'].sum()

1.0000000000000002

In [37]:
# Далее проведем проверку на согласованность, 
#для этого создадим из нашей полученной выше таблицы матрицы для дальнейшего их перемножения
df_new4 =df_new['mean']

# удаляем из нашей таблицы значениея столбцы mean
#df_new.pop('mean')

df_new1 =DF_1.loc[0]

df_new2 =DF_1.loc[1]

df_new3 =DF_1.loc[2]

In [38]:
#Проверка на согласованность
res1 = np.dot(df_new1,df_new4)
print(res1)
res2 = np.dot(df_new2,df_new4)
print(res2)
res3 = np.dot(df_new3,df_new4)
print(res3)

2.7883515452933816
0.7554190299991673
0.16153936301527372


In [40]:
sum_new = res1 + res2 + res3 
print(sum_new)

3.7053099383078227


In [42]:
# Вычислем значения CI, RI, CR
CI = (sum_new - 3)/(3-1)
print(CI)
RI = (1.98*(3-2))/3
print(RI)
CR = CI/RI
print(CR)

0.35265496915391137
0.66
0.5343257108392596


In [ ]:
# Получилось, что значение CR=0.53, что больше 0,1 значит матрицу нельзя считать согласованной
# Проделаем аналогичные действия с другими экспертами